In [13]:
import os
import cv2
import numpy as np
import gradio as gr

# 1) Import Keras (either from TF or standalone)
try:
    from tensorflow.keras.applications import VGG19
    from tensorflow.keras.layers      import GlobalAveragePooling2D, Dense, Dropout, Input
    from tensorflow.keras.models      import Model
    from tensorflow.keras.optimizers  import Adam
    print("Using tensorflow.keras")
except ModuleNotFoundError:
    from keras.applications import VGG19
    from keras.layers      import GlobalAveragePooling2D, Dense, Dropout, Input
    from keras.models      import Model
    from keras.optimizers  import Adam
    print("Using standalone keras")

# 2) Rebuild your trained model architecture
def build_detector(input_shape=(224,224,3),
                   freeze_upto=15,
                   dense_units=128,
                   dropout_rate=0.3,
                   lr=1e-4):
    base = VGG19(weights=None, include_top=False, input_tensor=Input(shape=input_shape))
    for layer in base.layers[:freeze_upto]:
        layer.trainable = False
    x = GlobalAveragePooling2D()(base.output)
    x = Dense(dense_units, activation="relu")(x)
    x = Dropout(dropout_rate)(x)
    outputs = Dense(2, activation="softmax")(x)
    model = Model(inputs=base.input, outputs=outputs)
    model.compile(
        optimizer=Adam(learning_rate=lr),
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model

# 3) Instantiate & load weights
model = build_detector()
weights_path = "best_model.h5"
if not os.path.isfile(weights_path):
    raise FileNotFoundError(f"Cannot find weights file: {weights_path}")
model.load_weights(weights_path, by_name=True)
print("✅ Weights loaded from", weights_path)

# 4) Video classification function
def classify_video(uploaded_file):
    """
    uploaded_file: Path to an uploaded video file.
    Returns "Real" or "Fake".
    """
    video_path = uploaded_file.name if hasattr(uploaded_file, "name") else uploaded_file
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        return "Error: cannot open video."

    fps = cap.get(cv2.CAP_PROP_FPS) or 1.0
    total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    duration = int(total / fps)

    fake_probs = []
    for sec in range(duration):
        cap.set(cv2.CAP_PROP_POS_MSEC, sec * 1000)
        ret, frame = cap.read()
        if not ret:
            break
        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (224,224))
        img = img.astype("float32") / 255.0
        img = np.expand_dims(img, 0)

        p = model.predict(img, verbose=0)
        fake_p = float(p[0][1]) if p.shape[-1] == 2 else float(p[0][0])
        fake_probs.append(fake_p)

    cap.release()
    if not fake_probs:
        return "Error: no frames extracted."

    count = 0 
    for prob in fake_probs:
        if prob >= 0.45:
            count += 1 
        
    print(fake_probs)
    return "Fake" if count > len(fake_probs) // 2 else "Real"

# 5) Gradio interface with a file uploader
iface = gr.Interface(
    fn=classify_video,
    inputs=gr.File(label="Upload Video (any format)"),
    outputs=gr.Textbox(label="Final Prediction"),
    title="Deepfake Video Detector",
    description="Upload a video file; the app samples 1 frame per second and labels it Real or Fake."
)

if __name__ == "__main__":
    iface.launch()


Using standalone keras
✅ Weights loaded from best_model.h5
* Running on local URL:  http://127.0.0.1:7865
* To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "c:\Users\adamy\OneDrive\Desktop\DeepFake_Numpy\tf-env\lib\site-packages\gradio\queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
  File "c:\Users\adamy\OneDrive\Desktop\DeepFake_Numpy\tf-env\lib\site-packages\gradio\route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
  File "c:\Users\adamy\OneDrive\Desktop\DeepFake_Numpy\tf-env\lib\site-packages\gradio\blocks.py", line 2146, in process_api
    result = await self.call_function(
  File "c:\Users\adamy\OneDrive\Desktop\DeepFake_Numpy\tf-env\lib\site-packages\gradio\blocks.py", line 1664, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
  File "c:\Users\adamy\OneDrive\Desktop\DeepFake_Numpy\tf-env\lib\site-packages\anyio\to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
  File "c:\Users\adamy\OneDrive\Desktop\DeepFake_Numpy\tf-en

In [20]:
import os
import cv2
import numpy as np
import gradio as gr
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Layer

# 1) Custom Cast layer for loading tf preprocess ops
class Cast(Layer):
    def __init__(self, dtype='float32', **kwargs):
        super().__init__(**kwargs)
        self._dtype = dtype
    def call(self, inputs):
        return tf.cast(inputs, self._dtype)
    def get_config(self):
        cfg = super().get_config()
        cfg.update({'dtype': self._dtype})
        return cfg

custom_objects = {'Cast': Cast}

# 2) Load your model
model_path = "best_model.h5"
if not os.path.exists(model_path):
    raise FileNotFoundError(f"Model file not found: {model_path}")

model = load_model(model_path, custom_objects=custom_objects, compile=False)
print("✅ Loaded model from", model_path)

# 3) Determine the expected input size (height, width)
# model.input_shape is typically (None, H, W, 3)
_, H, W, _ = model.input_shape
IMG_SIZE = (H, W)
print(f"ℹ️ Resizing inputs to {IMG_SIZE}")

# 4) Prediction function
def classify_image(img_pil):
    try:
        # Convert PIL → BGR numpy array
        img = cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR)
        # Resize to model’s expected dimensions
        img = cv2.resize(img, IMG_SIZE)

        # Normalize exactly as in training
        img = img.astype(np.float32) / 255.0
        img = np.expand_dims(img, 0)  # shape (1, H, W, 3)

        preds = model.predict(img, verbose=0)
        preds = np.squeeze(preds)

        # If you have a single sigmoid output
        if preds.ndim == 0:
            score = float(preds)
            label = "Fake" if score >= 0.5 else "Real"
            conf = score
        else:
            # softmax over 2 classes
            idx = int(np.argmax(preds))
            label = "Real" if idx == 1 else "Fake"
            conf = float(preds[idx])

        return f"{label} (confidence: {conf:.2f})"

    except Exception as e:
        return f"❌ Error during prediction:\n{type(e).__name__}: {e}"

# 5) Gradio app
iface = gr.Interface(
    fn=classify_image,
    inputs=gr.Image(label="Upload Image", type="pil"),
    outputs=gr.Textbox(label="Prediction"),
    title="Deepfake Image Detector",
    description=(
        "Resizes your upload to the model’s required input size, "
        "then classifies as Real or Fake."
    )
)

if __name__ == "__main__":
    iface.launch()


ModuleNotFoundError: No module named 'tensorflow.keras'